In [56]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px


In [57]:
path = 'https://www.ers.usda.gov/webdocs/DataFiles/51035/Fruit%20Prices%202020.csv?v=2687.3'
df = pd.read_csv(path)

In [58]:
df

,Fruit,Form,RetailPrice,RetailPriceUnit,Yield,CupEquivalentSize,CupEquivalentUnit,CupEquivalentPrice
0,Apples,Fresh,1.5193,per pound,0.90,0.2425,pounds,0.4094
1,"Apples, applesauce",Canned,1.0660,per pound,1.00,0.5401,pounds,0.5758
2,"Apples, ready-to-drink",Juice,0.7804,per pint,1.00,8.0000,fluid ounces,0.3902
3,"Apples, frozen concentrate",Juice,0.5853,per pint,1.00,8.0000,fluid ounces,0.2926
4,Apricots,Fresh,2.9665,per pound,0.93,0.3638,pounds,1.1603
...,...,...,...,...,...,...,...,...
57,Raspberries,Fresh,6.6391,per pound,0.96,0.3197,pounds,2.2107
58,Raspberries,Frozen,4.1877,per pound,1.00,0.3307,pounds,1.3849
59,Strawberries,Fresh,2.5800,per pound,0.94,0.3197,pounds,0.8774
60,Strawberries,Frozen,2.8189,per pound,1.00,0.3307,pounds,0.9322


In [104]:
app = Dash(__name__)

def find_cheapest_fruit(select_form):
    
    if select_form == 'allfruits':
        filtered_df = df

    else:
        filtered_df = df[df['Form']==select_form]  
    cheapest_fruit = filtered_df.loc[filtered_df['RetailPrice'].idxmin()]
    cheapest_fruit_name = cheapest_fruit['Fruit']
    cheapest_fruit_price = cheapest_fruit['RetailPrice']
    cheapest_fruit_weight = cheapest_fruit['RetailPriceUnit']
    
    return cheapest_fruit_name, cheapest_fruit_price, cheapest_fruit_weight

def find_expensive_fruit(select_form):
    
    if select_form == 'allfruits':
        filtered_df = df
    else:
        filtered_df = df[df['Form']==select_form]  
    expensive_fruit = filtered_df.loc[filtered_df['RetailPrice'].idxmax()]
    expensive_fruit_name = expensive_fruit['Fruit']
    expensive_fruit_price = expensive_fruit['RetailPrice']
    expensive_fruit_weight = expensive_fruit['RetailPriceUnit']
    
    return expensive_fruit_name, expensive_fruit_price, expensive_fruit_weight


app.layout = html.Div([
    html.H1('Fruits Vs Retail Prices',
            style= {'textAlign' : 'center'}),
    html.Br(),
    html.H3('Select a form of fruit',
               style={'textAlign' : 'center'}),
    dcc.Dropdown(id='mydropdown',
                 options =[
                      {'label': 'All Fruits', 'value': 'allfruits'},
                      *[
                      {'label': form, 'value' : form} for form in df['Form'].unique()
                      ]
                 ],
                 value = 'allfruits',
                 ),
    dcc.Graph(id= 'price-graph'),
    
    html.Br(),
    html.Div(id='cheapest_fruit', style={'textAlign' :'center'}),

    html.Br(),
    html.Div(id='expensive_fruit', style={'textAlign' :'center'})
])

@app.callback([Output('price-graph', 'figure'),
               Output('cheapest_fruit', 'children'),
               Output('expensive_fruit', 'children')],
              Input('mydropdown', 'value'))

def sync_input(select_form):
    if select_form == 'allfruits':
        filtered_df = df

    else:
        filtered_df = df[df['Form']==select_form]    
    
    fig = px.bar(filtered_df, x='Fruit', y='RetailPrice', title = f'Prices Trend for {select_form} Fruits')
    fig.update_yaxes(title_text = 'Retail Prices')
    fig.update_layout(title_x=0.5)
    
    cheapest_fruit_name, cheapest_fruit_price, cheapest_fruit_weight = find_cheapest_fruit(select_form)
    cheapest_fruit_info = f'Most cheapest fruit {select_form} fruit from is: {cheapest_fruit_name}, with a retail price of ${cheapest_fruit_price.round(2)}, {cheapest_fruit_weight}'

    expensive_fruit_name, expensive_fruit_price, expensive_fruit_weight = find_expensive_fruit(select_form)
    expensive_fruit_info = f'Most expensive fruit {select_form} fruit from is: {expensive_fruit_name}, with a retail price of ${expensive_fruit_price.round(2)}, {expensive_fruit_weight}'


    return fig, cheapest_fruit_info, expensive_fruit_info

if __name__ == '__main__':
    app.run_server(port=8051)
